In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import plotly.graph_objects as go
from datetime import datetime, timedelta

pd.options.display.float_format = '{:,.4f}'.format

# Inputs and Parameters
filename = 'copy_spx_quotedata.csv'

# Black-Scholes European-Options Gamma
def calcGammaEx(S, K, vol, T, r, q, optType, OI):
    if T == 0 or vol == 0:
        return 0
    dp = (np.log(S/K) + (r - q + 0.5*vol**2)*T) / (vol*np.sqrt(T))
    dm = dp - vol*np.sqrt(T) 
    gamma = np.exp(-q*T) * norm.pdf(dp) / (S * vol * np.sqrt(T))
    return OI * 100 * S * S * 0.01 * gamma 

# Read file and parse the first few lines to find the SPX spot and today's date
optionsFile = open(filename)
optionsFileData = optionsFile.readlines()
optionsFile.close()

spotPrice = float([line.split('Last: ')[1].split(',')[0].strip().replace(',', '') for line in optionsFileData if 'Last: ' in line][0])
todayDate = datetime.strptime([line.split('Date: ')[1].split(' at')[0].strip() for line in optionsFileData if 'Date: ' in line][0], '%B %d, %Y')

fromStrike = 0.8 * spotPrice
toStrike = 1.2 * spotPrice

# Get SPX Options Data
df = pd.read_csv(filename, sep=",", header=None, skiprows=4)
df.columns = ['ExpirationDate','Calls','CallLastSale','CallNet','CallBid','CallAsk','CallVol',
              'CallIV','CallDelta','CallGamma','CallOpenInt','StrikePrice','Puts','PutLastSale',
              'PutNet','PutBid','PutAsk','PutVol','PutIV','PutDelta','PutGamma','PutOpenInt']

df['ExpirationDate'] = pd.to_datetime(df['ExpirationDate'], format='%a %b %d %Y')
df['StrikePrice'] = df['StrikePrice'].astype(float)
df['CallIV'] = df['CallIV'].astype(float)
df['PutIV'] = df['PutIV'].astype(float)
df['CallGamma'] = df['CallGamma'].astype(float)
df['PutGamma'] = df['PutGamma'].astype(float)
df['CallOpenInt'] = df['CallOpenInt'].astype(float)
df['PutOpenInt'] = df['PutOpenInt'].astype(float)

# Filter for 1DTE options only (expiring tomorrow)
tomorrowDate = todayDate + timedelta(days=1)
df = df[df['ExpirationDate'].dt.date == tomorrowDate.date()]

# ---=== CALCULATE SPOT GAMMA ===---
df['CallGEX'] = df['CallGamma'] * df['CallOpenInt'] * 100 * spotPrice * spotPrice * 0.01
df['PutGEX'] = df['PutGamma'] * df['PutOpenInt'] * 100 * spotPrice * spotPrice * 0.01 * -1

df['TotalGamma'] = (df['CallGEX'] + df['PutGEX']) / 10**9
dfAgg = df.groupby(['StrikePrice']).agg({'TotalGamma': 'sum'})
strikes = dfAgg.index.values

# Define the range of strikes to display (300 points above and below SPX spot price)
lower_bound = spotPrice - 300
upper_bound = spotPrice + 300

# Filter dfAgg to include only strikes within the defined range
filtered_dfAgg = dfAgg[(dfAgg.index >= lower_bound) & (dfAgg.index <= upper_bound)]

# Filter the strikes array to match the filtered dataframe
filtered_strikes = filtered_dfAgg.index.values

# Chart 1: Absolute Gamma Exposure with Plotly (filtered by strike price range)
fig1 = go.Figure()

# Add Gamma Exposure bars
fig1.add_trace(go.Bar(
    x=filtered_strikes,
    y=filtered_dfAgg['TotalGamma'].to_numpy(),
    marker_color='blue',
    name="Gamma Exposure"
))

# Add SPX Spot line
fig1.add_vline(x=spotPrice, line=dict(color='red', width=2), name="SPX Spot")

# Layout settings
fig1.update_layout(
    title=f"Total Gamma: ${df['TotalGamma'].sum():.2f} Bn per 1% SPX Move",
    xaxis_title="Strike",
    yaxis_title="Spot Gamma Exposure ($ billions/1% move)",
    legend_title="Legend",
    showlegend=True,
    width=900,
    height=500,
    xaxis=dict(range=[lower_bound, upper_bound])  # Set x-axis to only show the filtered range
)

fig1.show()

# ---=== CALCULATE GAMMA PROFILE FOR 1DTE ===---
levels = np.linspace(fromStrike, toStrike, 60)
totalGamma = []

# For each spot level, calculate gamma exposure at that point using 1DTE data
for level in levels:
    df['callGammaEx'] = df.apply(lambda row: calcGammaEx(level, row['StrikePrice'], row['CallIV'], 
                                                         1/365, 0, 0, "call", row['CallOpenInt']), axis=1)
    df['putGammaEx'] = df.apply(lambda row: calcGammaEx(level, row['StrikePrice'], row['PutIV'], 
                                                        1/365, 0, 0, "put", row['PutOpenInt']), axis=1)    

    totalGamma.append(df['callGammaEx'].sum() - df['putGammaEx'].sum())

totalGamma = np.array(totalGamma) / 10**9  # Normalize to billions

# Find Gamma Flip Point
zeroCrossIdx = np.where(np.diff(np.sign(totalGamma)))[0]

if zeroCrossIdx.size > 0:
    gammaFlipPoint = np.interp(0, totalGamma[zeroCrossIdx[0]:zeroCrossIdx[0]+2], levels[zeroCrossIdx[0]:zeroCrossIdx[0]+2])
else:
    gammaFlipPoint = None  # Handle the case where no zero-crossing is found

# Chart 2: Gamma Exposure Profile for 1DTE with dual background colors and hover functionality
fig2 = go.Figure()

# Add Gamma Exposure line
fig2.add_trace(go.Scatter(
    x=levels,
    y=totalGamma,
    mode='lines',
    name="1DTE Gamma Exposure"
))

# Add SPX Spot line as a vertical scatter line with hover functionality
fig2.add_trace(go.Scatter(
    x=[spotPrice, spotPrice],
    y=[min(totalGamma) - 0.5, max(totalGamma) + 0.5],
    mode='lines',
    line=dict(color='red', width=2),
    name=f"SPX Spot: {spotPrice:.0f}",
    hovertemplate=f"SPX Spot Price: {spotPrice:.0f}"
))

# Add Gamma Flip Point line as a vertical scatter line with hover functionality
if gammaFlipPoint is not None:
    fig2.add_trace(go.Scatter(
        x=[gammaFlipPoint, gammaFlipPoint],
        y=[min(totalGamma) - 0.5, max(totalGamma) + 0.5],
        mode='lines',
        line=dict(color='black', width=2),
        name=f"Gamma Flip Point: {gammaFlipPoint:.0f}",
        hovertemplate=f"Gamma Flip Point: {gammaFlipPoint:.0f}"
    ))

# Add shaded background on the left (before Gamma Flip Point)
fig2.add_shape(type="rect",
               x0=fromStrike, x1=gammaFlipPoint if gammaFlipPoint is not None else spotPrice,
               y0=-10, y1=10,
               fillcolor="rgba(255, 192, 203, 0.2)",  # light red (similar to pink)
               line_width=0)

# Add shaded background on the right (after Gamma Flip Point)
fig2.add_shape(type="rect",
               x0=gammaFlipPoint if gammaFlipPoint is not None else spotPrice, x1=toStrike,
               y0=-10, y1=10,
               fillcolor="rgba(144, 238, 144, 0.2)",  # light green
               line_width=0)

# Layout settings with dual background and hover-enabled lines
fig2.update_layout(
    title=f"Gamma Exposure Profile, SPX (1DTE), {todayDate.strftime('%d %b %Y')}",
    xaxis_title="Index Price",
    yaxis_title="Gamma Exposure ($ billions/1% move)",
    width=900,
    height=500,
    xaxis=dict(range=[fromStrike, toStrike]),
    yaxis=dict(range=[min(totalGamma) - 0.5, max(totalGamma) + 0.5]),
    plot_bgcolor='rgba(255, 255, 255, 1)',  # White background
)

fig2.show()


# Define the range of strikes to display (300 points above and below SPX spot price)
lower_bound = spotPrice - 300
upper_bound = spotPrice + 300

# Filter dfAgg to include only strikes within the defined range
filtered_dfAgg = dfAgg[(dfAgg.index >= lower_bound) & (dfAgg.index <= upper_bound)]

# Filter the strikes array to match the filtered dataframe
filtered_strikes = filtered_dfAgg.index.values

# Filter top 5 below and above SPX spot price within the defined range
top_5_below = filtered_dfAgg[filtered_dfAgg.index < spotPrice].nsmallest(5, 'TotalGamma')
top_5_above = filtered_dfAgg[filtered_dfAgg.index > spotPrice].nlargest(5, 'TotalGamma')

# Define the range of strikes to display (300 points above and below SPX spot price)
lower_bound = spotPrice - 300
upper_bound = spotPrice + 300

# Filter dfAgg to include only strikes within the defined range
filtered_dfAgg = dfAgg[(dfAgg.index >= lower_bound) & (dfAgg.index <= upper_bound)]

# Filter the strikes array to match the filtered dataframe
filtered_strikes = filtered_dfAgg.index.values

# Get the top 10 highest and lowest gamma exposure levels within the defined range
top_10_gamma = filtered_dfAgg.nlargest(10, 'TotalGamma')
lowest_10_gamma = filtered_dfAgg.nsmallest(10, 'TotalGamma')

# Chart 3: Highlight Top 10 and Lowest 10 Gamma Exposure Levels with Plotly (filtered by strike price range)
fig3 = go.Figure()

# Add Gamma Exposure bars (without strike labels on the bars themselves)
fig3.add_trace(go.Bar(
    x=filtered_strikes,
    y=filtered_dfAgg['TotalGamma'].to_numpy(),
    marker_color='blue',
    name="Gamma Exposure"
))

# Highlight and label the top 10 gamma exposure levels
for idx, row in top_10_gamma.iterrows():
    fig3.add_trace(go.Scatter(
        x=[row.name], y=[row['TotalGamma']], mode='markers',
        marker=dict(color='green', size=10),
        name=f"Top Strike: {row.name:.0f}"
    ))

# Highlight and label the lowest 10 gamma exposure levels
for idx, row in lowest_10_gamma.iterrows():
    fig3.add_trace(go.Scatter(
        x=[row.name], y=[row['TotalGamma']], mode='markers',
        marker=dict(color='blue', size=10),
        name=f"Low Strike: {row.name:.0f}"
    ))

# Add SPX Spot line
fig3.add_vline(x=spotPrice, line=dict(color='red', width=2), name=f"SPX Spot: {spotPrice:.0f}")

# Layout settings
fig3.update_layout(
    title="Top 10 Highest and Lowest Gamma Exposure Levels (1DTE)",
    xaxis_title="Strike",
    yaxis_title="Spot Gamma Exposure ($ billions/1% move)",
    width=900,
    height=500,
    xaxis=dict(range=[lower_bound, upper_bound]),  # Set x-axis to only show the filtered range
    showlegend=True  # Show the legend with the strikes labeled
)

fig3.show()
